In [26]:
#### I want to create a new dataframe to be modeled (ETL)
####
#### Steps
####  1) First pull the original csv file created from extract_part function and add team index
####  2) Make variables that will go in to the final table
####  3) Join the pieces into a final table (800 rows)
####  4) Partition the final table into two, so that each row becomes independent of each other and write csv

#### Prereq
####  1) csv file from extract_part function


In [27]:
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np
import matplotlib.pyplot as plt
import csv
%matplotlib inline


#### step 1) First pull the original csv file created from extract_part function and add team index
#trainset
hijae = pd.read_csv("hijaedata.csv")[:-1]

#testsets
puree = pd.read_csv("pureedata.csv")[:-1]
hikari= pd.read_csv("hikaridata.csv")[:-1]
king = pd.read_csv("king.csv")[:-1]
runes = pd.read_csv("runes.csv")[:-1]

####  adding indexes for group by sum

def addindex(dfr):
    index_col=[]
    for i in range(0, len(dfr)/5):
        index_col.append(i)
        index_col.append(i)
        index_col.append(i)
        index_col.append(i)
        index_col.append(i)
    dfr['index']=index_col

addindex(hijae)
addindex(hikari)
addindex(puree)
addindex(king)
addindex(runes)

# quick qa
hijae.head()

,part_id,match_id,winner,side,role,game_cnt,win,loss,win_rate,kda,kpg,dpg,apg,gpg,ddpg,dtpg,tpg,cs,index
0,0,2005044165,True,blue,Role.carry,4,3,1,0.750000,3.846154,6.500000,3.250000,6.000000,12775.00000,169045.75000,15918.00000,2.000000,202.000000,0
1,1,2005044165,True,blue,Role.support,14,8,6,0.571429,4.271186,4.142857,4.214286,13.857143,10110.64286,44714.28571,21850.71429,0.500000,38.428571,0
2,2,2005044165,True,blue,Role.solo,4,2,2,0.500000,2.500000,8.250000,5.000000,4.250000,12721.50000,160485.00000,17489.75000,0.750000,204.000000,0
3,3,2005044165,True,blue,Role.none,285,150,135,0.526316,2.494467,8.936842,6.024561,6.091228,13110.17193,171587.54390,30422.98246,0.459649,65.508772,0
4,4,2005044165,True,blue,Role.solo,78,38,40,0.487179,2.859116,5.192308,4.641026,8.076923,11772.85897,132691.05130,31927.64103,0.974359,198.038462,0


In [25]:
####  2) Make variables that will go in to the final table

#아.. 클래스써야겠다.. 나중에 하자...;;

def hijaeframe(dfr):
    ##groupby index and sum
    partitioned=dfr.iloc[::,5:]
    summed=partitioned.groupby('index').sum()
    
    #final dataframe
    loldf= pd.DataFrame()
    loldf['winner']=[dfr['winner'][i] for i in range(0,len(dfr), 5)]
    
    #side - dummy variable
    side_dummy = pd.get_dummies([dfr['side'][i] for i in range(1,len(dfr), 5)], prefix='side')
    loldf[[i for i in side_dummy.columns]] = side_dummy
    
    #rest of the variable creation + addition
    loldf[[i for i in summed.columns]]=summed[[i for i in summed.columns]]
    loldf['newwin']=(loldf['win']/loldf['game_cnt'])*100  ##
    loldf['topsin']=loldf['tpg']*(loldf['kpg']+loldf['apg'])
    loldf['ggg']=( (loldf['ddpg']-(loldf['cs']*750))/ (loldf['dpg']))
    loldf['kapg']=loldf['kpg']+loldf['apg']
    return loldf

#### 3) divide the dataframe into blue side and red side

model_blue = hijaeframe(runes).iloc[0::2, :]
model_blue.head(10)
model_blue.to_csv("runes_blue.csv")


model_blue = hijaeframe(runes).iloc[1::2, :]
model_blue.to_csv("runes_red.csv")
#model_blue.head()
#hijaeframe(puree).iloc[1::2, :].to_csv("puree_red.csv")
#hijaeframe(hikari).iloc[1::2, :].to_csv("hikari_red.csv")


In [30]:
##just testing something out

def hijaelogr(dfr):
    logr = sm.Logit(dfr['winner'], dfr[['tpg', 'dpg', 'win_rate', 'topsin', 'kapg']])
    return logr

###  make this into class-objects ###
### 
hijaem = hijaelogr(model_blue).fit()
print hijaem.summary()
print np.exp(hijaem.params)

#print np.exp(hijaem.conf_int())

Optimization terminated successfully.
         Current function value: 0.454958
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                 winner   No. Observations:                  150
Model:                          Logit   Df Residuals:                      145
Method:                           MLE   Df Model:                            4
Date:                Wed, 30 Mar 2016   Pseudo R-squ.:                  0.3423
Time:                        13:40:00   Log-Likelihood:                -68.244
converged:                       True   LL-Null:                       -103.76
                                        LLR p-value:                 1.376e-14
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
tpg           -2.2790      0.771     -2.955      0.003        -3.791    -0.767
dpg            0.0057      0.